# Point Inside Circle Neural Network

This notebook implements a simple feed‑forward neural network (Multi‑Layer Perceptron) that predicts whether a point with coordinates $(x, y)$ lies inside a circle of radius 5 centered at $(0, 0)$.

A point belongs to the circle if the following condition holds:

\[
x^2 + y^2 \le 25
\]

The goal is to:

1. Generate a synthetic dataset of points and their labels (inside / outside the circle).
2. Build a neural network model using TensorFlow / Keras.
3. Train the model using the backpropagation algorithm.
4. Evaluate the model on a test set.
5. Test the trained model on several example points.


## 1. Imports

In this section we import all necessary libraries:

- `numpy` — for numerical operations and array handling.
- `matplotlib` — for simple visualizations.
- `sklearn.model_selection.train_test_split` — to split data into training and test sets.
- `tensorflow` and `tensorflow.keras` — to define and train the neural network.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models

# For reproducibility of random numbers
np.random.seed(42)
tf.random.set_seed(42)


## 2. Data generation

We generate points uniformly in the square $[-6, 6] \times [-6, 6]$.  
For each point $(x, y)$ we assign a label:

- `1.0` — the point is inside or on the boundary of the circle (positive class);
- `0.0` — the point is outside the circle (negative class).

The labeling rule is based on the inequality:

\[
x^2 + y^2 \le r^2, \quad r = 5.
\]


In [ ]:
# Total number of points to generate
N = 20000

# Generate N two-dimensional points in the range [-6, 6] x [-6, 6]
X = np.random.uniform(-6, 6, size=(N, 2))


def point_in_circle(x: float, y: float, r: float = 5.0) -> float:
    """Return 1.0 if (x, y) lies inside or on a circle of radius r, else 0.0.

    Mathematically, a point is inside a circle of radius r centered at (0, 0)
    if x**2 + y**2 <= r**2.
    """
    return 1.0 if x**2 + y**2 <= r**2 else 0.0


# Build the vector of labels for all points
y = np.array([point_in_circle(x, y) for x, y in X], dtype=np.float32)

print("Sample point:", X[0])
print("Its label:", y[0])


### Optional: visualize the data and the circle

To get an intuition about the task, we can plot a subset of points and the circle of radius 5.
Points inside the circle are shown in one color, points outside — in another.


In [ ]:
# Take a subset of points for visualization (too many points would make the plot heavy)
idx = np.random.choice(len(X), size=1000, replace=False)
X_vis = X[idx]
y_vis = y[idx]

inside = X_vis[y_vis == 1.0]
outside = X_vis[y_vis == 0.0]

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(outside[:, 0], outside[:, 1], s=5, label="outside")
ax.scatter(inside[:, 0], inside[:, 1], s=5, label="inside")


circle = plt.Circle((0, 0), 5, fill=False)
ax.add_artist(circle)

ax.set_aspect('equal', 'box')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Points inside and outside the circle (r = 5)')
ax.legend()
plt.show()


## 3. Train–test split

We split the dataset into two parts:

- **training set** (80%) — used to fit the model;
- **test set** (20%) — used to evaluate model performance on unseen data.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])


## 4. Neural network architecture

We build a simple fully connected neural network (Multi‑Layer Perceptron):

- Input layer: 2 features (x and y).
- Hidden layer 1: 16 neurons, ReLU activation.
- Hidden layer 2: 16 neurons, ReLU activation.
- Output layer: 1 neuron, Sigmoid activation.

**ReLU** is used in hidden layers because it works well for nonlinear problems and helps avoid the vanishing gradient problem.  
**Sigmoid** is used in the output layer because the task is binary classification and we interpret its output as a probability.


In [ ]:
model = models.Sequential([
    # Input layer: expects vectors of size 2 (x and y)
    layers.Input(shape=(2,)),

    # First hidden layer with 16 neurons and ReLU activation
    layers.Dense(16, activation='relu'),

    # Second hidden layer with 16 neurons and ReLU activation
    layers.Dense(16, activation='relu'),

    # Output layer: 1 neuron with Sigmoid, outputs probability in [0, 1]
    layers.Dense(1, activation='sigmoid')
])

# Compile the model: choose optimizer, loss, and metrics
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Display a summary of the model
model.summary()


## 5. Training the model (Backpropagation)

When we call `model.fit`, Keras performs the following steps internally:

1. **Forward pass** — computes predictions for a batch of input data.
2. **Loss computation** — compares predictions with true labels using the loss function.
3. **Backward pass (Backpropagation)** — computes gradients of the loss with respect to all weights using the chain rule and derivatives of activation functions.
4. **Weight update** — updates each weight according to the optimization algorithm (here, Adam).

We track both training and validation metrics during training.


In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,  # Use 20% of the training data for validation
    epochs=20,
    batch_size=32,
    verbose=1
)


### 5.1. Training curves

Let us plot the training and validation loss and accuracy to see how the model behaves during training.


In [ ]:
# Plot loss curves
plt.figure()
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# Plot accuracy curves
plt.figure()
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()


## 6. Evaluation on the test set

Now we evaluate the trained model on the test dataset that was not used during training.


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_acc:.4f}")


## 7. Predicting for individual points

Finally, we define a small helper function that:

1. Takes coordinates `(x, y)`.
2. Runs the model to obtain the predicted probability that the point lies inside the circle.
3. Converts this probability into a class label using a threshold (by default 0.5).


In [ ]:
def predict_point(x: float, y: float, threshold: float = 0.5) -> int:
    """Predict whether a single point (x, y) is inside the circle.

    The function:
    - builds a batch with a single point;
    - uses the trained model to compute the probability that the point is inside;
    - compares the probability with the given threshold and returns:
        1 if prob >= threshold,
        0 otherwise.
    """
    point = np.array([[x, y]], dtype=np.float32)
    prob = model.predict(point, verbose=0)[0, 0]
    predicted_class = 1 if prob >= threshold else 0
    print(f"Point ({x:.2f}, {y:.2f}) -> prob_inside = {prob:.3f}, predicted_class = {predicted_class}")
    return predicted_class


# Test the function on a few example points
predict_point(1, 1)    # inside
predict_point(5, 0)    # on the boundary
predict_point(6, 0)    # outside
predict_point(4, 4)    # outside
